# Geospatial Python
## Crop raster data with rioxarray and geopandas
Setup: https://carpentries-incubator.github.io/geospatial-python/index.html

Instruction: https://carpentries-incubator.github.io/geospatial-python/08-crop-raster-data.html

Objectives:
* Crop raster data with a bounding box.
* Crop raster data with a polygon.
* Match two raster datasets in different CRS.

We will use the results of the satellite image search 'search.json', which was generated in 'Geospatial Python 05 - Intro.ipynb'.
And the cropped fields polygons 'fields_cropped.shp', which was generated in 'Geospatial Python 07 - Vector Data - complete.ipynb'.

Before executing the code cells, be sure to replace the "_____" as appropriate

In [ ]:
# first import necessary libraries
import pystac
import rioxarray

# (lazy) Load 6th image from search result and inspect the shape
items = pystac.ItemCollection.from_file("_____")
raster = rioxarray.open_rasterio(items[5].assets["_____"].href) # Select a true color image "visual"
print(raster.shape)

In [ ]:
# Get the overview asset
# Note: “Overviews” are precomputed lower resolution representations of a raster, part of COG standard
raster_overview = rioxarray.open_rasterio(items[5].assets["_____"].href, overview_level=3)
print(raster_overview.shape)

# Visualize it
raster_overview.plot.imshow(figsize=(8,8))

In [ ]:
# Check the coordinate system of the raster
# this requires pyproj
from pyproj import CRS

CRS(raster.rio.crs)

In [ ]:
# Load and check the coordinate system of the 'fields_cropped.shp' file
import geopandas as gpd

# Load the polygons of the crop fields
fields = gpd.read_file("fields_cropped.shp")

# Check the coordinate reference system (crs)
fields."_____"

In [ ]:
# Convert the 'fields' CRS to the CRS of 'raster'
fields = fields.to_crs(raster.rio."_____")

## Crop raster data with a bounding box

In [ ]:
# Crop the raster with the bounding box (total_bounds property)
raster_clip_box = raster.rio.clip_box(*fields."_____")
# note the * unpacks the array
# inspect the shape
print(raster_clip_box.shape)

In [ ]:
# visualize the result
raster_clip_box.plot.imshow(figsize=(8,8))

In [ ]:
# save the cropped tif ("raster_clip.tif")
raster_clip_box.rio.to_raster("_____")

## Crop raster data with polygons

In [ ]:
# crop the image to the exact field boundaries
raster_clip_fields = raster_clip_box.rio.clip(fields['geometry'])

In [ ]:
# visualize the result
raster_clip_fields.plot.imshow(figsize=(8,8))

In [ ]:
''' Exercise: Crop raster data with a specific "crop code"

In the column “gewascode” (translated as “crop code”) of fields, you can find the code representing the types of plants grown in each field. 
Can you:

- Select the fields with “gewascode” equal to 257;
- Crop the raster raster_clip_box with the selected fields;
* Visualize the cropped image.

'''
mask = fields['gewascode']=="_____"
fields_gwascode = fields.where(mask)
fields_gwascode = fields_gwascode.dropna()
raster_clip_fields_gwascode = raster_clip_box.rio.clip(fields_gwascode['geometry'])
raster_clip_fields_gwascode.plot.imshow(figsize=(8,8))

## Crop raster data using reproject_match() function

In [ ]:
# Reproject 'raster_clip_fields_gwascode' to make the CRS different from the "raster"
# Use EPSG:28992
raster_clip_fields_gwascode = raster_clip_fields_gwascode.rio.reproject("_____")
CRS(raster_clip_fields_gwascode.rio.crs)

In [ ]:
# check the CRS of raster_clip_box
CRS(raster_clip_box.rio.crs)

In [ ]:
# crop raster_clip_box image agaist raster_clip_fields_gwascode with different projection
raster_reproject_match = raster_clip_box.rio.reproject_match(raster_clip_fields_gwascode)

#visualize the result
raster_reproject_match.plot.imshow(figsize=(8,8))

In [ ]:
# Try going the other way to expand the bounds
raster_reproject_match = raster_clip_fields_gwascode.rio.reproject_match(raster_clip_box)
raster_reproject_match.plot.imshow(figsize=(8,8))